## Célula 1
Nesta etapa centralizamos todas as bibliotecas usadas ao longo do projeto e carregamos o dataset.

Seguir esse caminho garante:

*   melhor organização do notebook
*   facilidade de rastreamento de dependências
*   transparência sobre quais ferramentas são necessárias para execução

Também realizamos uma primeira inspeção do dataframe (head, info, describe) para:


*   entender o formato dos dados
*   confirmar tipos de variáveis
*   verificar possíveis problemas como ausências, tipos inadequados ou dispersão inesperada

Essa visão inicial fundamenta todas as decisões posteriores de limpeza e modelagem.

In [17]:
# CÉLULA 1 - IMPORTS E CARREGAMENTO DO CSV

import pandas as pd
import numpy as np
import xgboost as xgb
import gradio as gr
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from scipy.optimize import minimize

# Caminho do CSV - ajuste se necessário
DATA_PATH = 'banco_solos_integrado_1000_linhas_v2.csv'

# Carregar o dataset
df = pd.read_csv(DATA_PATH)

# Manter uma cópia do dataframe original (antes do one-hot) — importante para buscar medianas e opções de categorias
df_original = df.copy()

print("Primeiras 5 linhas:")
print(df.head())

print("\nInformações do dataframe:")
df.info()

print("\nEstatísticas descritivas:")
display(df.describe())



Primeiras 5 linhas:
   id  cultura  tipo_solo    ph  umidade_pct  irrigacao_horas       n_mgkg  \
0   1  laranja  misturado  4.98         29.4              4.1  1156.398451   
1   2     cana  misturado  6.31         20.5              2.0  1026.618987   
2   3  laranja    massape  5.45         27.7              0.4  1001.713175   
3   4  laranja        NaN  7.10         12.0              5.8  1226.116394   
4   5     cana    arenoso  7.48         31.0              4.5   691.909007   

      p_mgkg      k_mgkg  producao_ton_ha  
0  41.753934         NaN        32.656532  
1  49.053094  210.538752        84.332550  
2  25.122190  122.188102        27.831272  
3  53.160568  196.893933        27.564167  
4  73.674936  125.874616       104.762201  

Informações do dataframe:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id               

,id,ph,umidade_pct,irrigacao_horas,n_mgkg,p_mgkg,k_mgkg,producao_ton_ha
count,1000.000000,978.000000,989.000000,981.000000,987.000000,982.000000,980.000000,983.000000
mean,500.500000,5.978517,22.028109,4.907645,905.926218,44.619928,151.950851,40.751716
std,288.819436,0.860170,7.674401,2.913486,193.519797,14.834247,58.350898,35.679074
min,1.000000,4.500000,8.000000,0.000000,372.806426,5.000000,20.000000,2.169159
25%,250.750000,5.210000,15.800000,2.300000,776.864413,34.253987,111.036917,3.582691
50%,500.500000,5.990000,22.400000,4.900000,900.142024,45.259934,151.024496,29.235426
75%,750.250000,6.740000,28.600000,7.500000,1035.521515,54.737980,191.136351,79.201580
max,1000.000000,7.500000,35.000000,10.000000,1631.540361,87.613296,365.968007,127.667406


## Célula 2
Antes de modelar, asseguramos que não existam valores vazios na base, pois:

*   algoritmos de machine learning não aceitam NaN
*   dados faltantes podem distorcer os resultados

Optamos por:

*   moda para variáveis categóricas, pois resgata o comportamento mais frequente
*   mediana para variáveis numéricas, pois é mais robusta em relação a outliers que a média

Com isso, garantimos consistência do dataset sem criar valores artificiais que possam afetar o comportamento dos modelos.

In [20]:
# CÉLULA 2 - TRATAMENTO DE VALORES AUSENTES

categorical_cols_with_na = ['cultura', 'tipo_solo']
numerical_cols_with_na = ['ph', 'umidade_pct', 'irrigacao_horas',
                          'n_mgkg', 'p_mgkg', 'k_mgkg', 'producao_ton_ha']

# Preencher categóricas com a moda
for col in categorical_cols_with_na:
    if df[col].isnull().any():
        df[col] = df[col].fillna(df[col].mode()[0])

# Preencher numéricas com a mediana
for col in numerical_cols_with_na:
    if df[col].isnull().any():
        df[col] = df[col].fillna(df[col].median())

print("\nValores ausentes após preenchimento:")
print(df.isnull().sum())





Valores ausentes após preenchimento:
id                 0
cultura            0
tipo_solo          0
ph                 0
umidade_pct        0
irrigacao_horas    0
n_mgkg             0
p_mgkg             0
k_mgkg             0
producao_ton_ha    0
dtype: int64


## Célula 3
Convertendo cultura e tipo de solo em variáveis binárias via get_dummies, possibilitamos:


* que modelos numéricos compreendam variáveis categóricas
* preservação das relações de cada classe com o resultado

Também optamos por drop_first=True para evitar multicolinearidade, tornando o modelo mais estável e interpretável.

In [21]:
# CÉLULA 3 - ONE-HOT ENCODING

df = pd.get_dummies(df, columns=['cultura', 'tipo_solo'], drop_first=True, dtype=int)

print("Primeiras 5 linhas após one-hot:")
print(df.head())


Primeiras 5 linhas após one-hot:
   id    ph  umidade_pct  irrigacao_horas       n_mgkg     p_mgkg      k_mgkg  \
0   1  4.98         29.4              4.1  1156.398451  41.753934  151.024496   
1   2  6.31         20.5              2.0  1026.618987  49.053094  210.538752   
2   3  5.45         27.7              0.4  1001.713175  25.122190  122.188102   
3   4  7.10         12.0              5.8  1226.116394  53.160568  196.893933   
4   5  7.48         31.0              4.5   691.909007  73.674936  125.874616   

   producao_ton_ha  cultura_laranja  cultura_soja  tipo_solo_argiloso  \
0        32.656532                1             0                   0   
1        84.332550                0             0                   0   
2        27.831272                1             0                   0   
3        27.564167                1             0                   0   
4       104.762201                0             0                   0   

   tipo_solo_latossolo  tipo_solo_massape

## Célula 4
Aqui definimos X (entradas) e y (resultado), removendo:

*   a coluna alvo
*   identificadores como id, pois não contribuem para o aprendizado

Fazemos o train_test_split para:

*   simular como o modelo performaria com dados nunca vistos
*   evitar overfitting
*   manter rigor no processo de avaliação

Essa separação garante que o desempenho medido seja mais realista e confiável.

In [22]:
# CÉLULA 4 - FEATURES E SPLIT

target_column = 'producao_ton_ha'

X = df.drop(columns=[col for col in ['id', target_column] if col in df.columns])
y = df[target_column]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print(f"X_train: {X_train.shape}, X_test: {X_test.shape}")
print(f"y_train: {y_train.shape}, y_test: {y_test.shape}")


X_train: (800, 12), X_test: (200, 12)
y_train: (800,), y_test: (200,)


## Célula 5
Testamos quatro algoritmos supervisionados: Linear Regression, Random Forest, Gradient Boosting e XGBoost.

Esse conjunto misturam modelos lineares simples e ensembles não lineares de grande capacidade.

Realizamos as análises de desempenho MAE,MSE, RMSE e R².

Comparar múltiplos algoritmos aumenta a confiabilidade do resultado, pois permite escolher aquele que melhor se adapta ao padrão real dos dados.

In [23]:
# CÉLULA 5 - TREINAMENTO DE MODELOS

models_to_train = {
    'LinearRegression': LinearRegression(),
    'RandomForestRegressor': RandomForestRegressor(random_state=42),
    'GradientBoostingRegressor': GradientBoostingRegressor(random_state=42),
    'XGBRegressor': xgb.XGBRegressor(random_state=42)
}

performance = {}
trained_models = {}

for name, model in models_to_train.items():
    model.fit(X_train, y_train)
    trained_models[name] = model

    pred = model.predict(X_test)

    mae = mean_absolute_error(y_test, pred)
    mse = mean_squared_error(y_test, pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, pred)

    performance[name] = {'MAE': mae, 'MSE': mse, 'RMSE': rmse, 'R2': r2}

performance


{'LinearRegression': {'MAE': 6.285930240294183,
  'MSE': 110.2952540635683,
  'RMSE': 10.502154734318491,
  'R2': 0.9132433518335449},
 'RandomForestRegressor': {'MAE': 6.54514821339872,
  'MSE': 133.66659854614255,
  'RMSE': 11.56142718465772,
  'R2': 0.8948597910206463},
 'GradientBoostingRegressor': {'MAE': 6.626743329726626,
  'MSE': 122.68158568622182,
  'RMSE': 11.076171977999522,
  'R2': 0.9035004429134543},
 'XGBRegressor': {'MAE': 7.674577445477233,
  'MSE': 156.02211099080301,
  'RMSE': 12.490881113468458,
  'R2': 0.8772752689647435}}

## Célula 6 – Modelo para otimização

Nesta célula, definimos o modelo usado para otimização no Gradio.

Como nosso dataset não funciona bem com modelos lineares, optamos por usar RandomForestRegressor, que captura relações não lineares e gera previsões mais realistas.

Quando o modelo é RandomForest, criamos automaticamente uma função de Random Search, que encontra combinações ideais de PH, umidade, irrigação e fertilizantes de forma rápida e segura, sem travar o notebook.

Esse processo garante:

* Rastreabilidade – sabemos qual modelo está sendo usado.
* Reprodutibilidade – resultados consistentes com semente fixa.
* Flexibilidade – basta trocar optimization_model_name para outro modelo compatível sem alterar a Célula 7.
* Uso do modelo mais adequado – evita problemas de modelos lineares e gera recomendações reais.



In [32]:
# CÉLULA 6 - DEFINIÇÃO DO MODELO PARA OTIMIZAÇÃO E RANDOM SEARCH
optimization_model_name = 'RandomForestRegressor'  # Mude aqui se quiser outro modelo
optimization_model = trained_models[optimization_model_name]

print(f"Modelo de otimização: {optimization_model_name}")
print(performance[optimization_model_name])

# Features a serem otimizadas
optimization_features = ['ph', 'umidade_pct', 'irrigacao_horas', 'n_mgkg', 'p_mgkg', 'k_mgkg']
feature_order = X_train.columns.tolist()

# Limites baseados no dataset
feature_bounds = {f: {'min': df[f].min(), 'max': df[f].max()} for f in optimization_features}

# Se for RandomForest, define Random Search
if optimization_model_name == 'RandomForestRegressor':
    import numpy as np


    def random_search_optimization(cultura, tipo_solo, n_iter=200):
        # Colunas one-hot fixas
        fixed_values = {col: 0 for col in feature_order if col.startswith('cultura_') or col.startswith('tipo_solo_')}
        if cultura != 'milho':
            fixed_values[f'cultura_{cultura}'] = 1
        if tipo_solo != 'arenoso':
            fixed_values[f'tipo_solo_{tipo_solo}'] = 1

        best_prod = -np.inf
        best_params = None

        for _ in range(n_iter):
            inp = {f: np.random.uniform(feature_bounds[f]['min'], feature_bounds[f]['max']) for f in
                   optimization_features}
            inp.update(fixed_values)
            df_inp = pd.DataFrame([inp], columns=feature_order)
            prod = optimization_model.predict(df_inp)[0]
            if prod > best_prod:
                best_prod = prod
                best_params = inp

        best_params['Produção Máxima (ton/ha)'] = best_prod
        return best_params




Modelo de otimização: RandomForestRegressor
{'MAE': 6.54514821339872, 'MSE': 133.66659854614255, 'RMSE': 11.56142718465772, 'R2': 0.8948597910206463}


## Célula 7 – Interface de otimização no Gradio

Nesta célula, criamos uma interface interativa que permite ao usuário escolher cultura e tipo de solo e recebe os valores ideais de PH, umidade, irrigação e fertilizantes, junto com a produção máxima prevista.

Essa abordagem transforma o notebook em uma ferramenta prática de decisão, mantendo flexibilidade para trocar o modelo de otimização sem alterar a lógica da interface.

In [33]:
#CÉLULA 7 - INTERFACE GRADIO

import gradio as gr

# Dropdowns
cultura_options = df.filter(like='cultura_').columns.str.replace('cultura_', '').tolist()
cultura_options = ['milho'] + [c for c in cultura_options if c != 'milho']

tipo_solo_options = df.filter(like='tipo_solo_').columns.str.replace('tipo_solo_', '').tolist()
tipo_solo_options = ['arenoso'] + [s for s in tipo_solo_options if s != 'arenoso']

def otimizar_parametros(cultura, tipo_solo):
    # Chama Random Search se for RandomForest
    if optimization_model_name == 'RandomForestRegressor':
        resultado = random_search_optimization(cultura, tipo_solo)
    else:
        # Para outros modelos, implementar outro método de otimização
        resultado = {}  # placeholder

    return (round(resultado['ph'],2),
            round(resultado['umidade_pct'],2),
            round(resultado['irrigacao_horas'],2),
            round(resultado['n_mgkg'],2),
            round(resultado['p_mgkg'],2),
            round(resultado['k_mgkg'],2),
            round(resultado['Produção Máxima (ton/ha)'],2))

# Interface Gradio
iface = gr.Interface(
    fn=otimizar_parametros,
    inputs=[
        gr.Dropdown(cultura_options, label="Cultura"),
        gr.Dropdown(tipo_solo_options, label="Tipo de Solo")
    ],
    outputs=[
        gr.Textbox(label="PH ideal"),
        gr.Textbox(label="Umidade ideal (%)"),
        gr.Textbox(label="Irrigação ideal (horas)"),
        gr.Textbox(label="Nitrogênio ideal (mg/kg)"),
        gr.Textbox(label="Fósforo ideal (mg/kg)"),
        gr.Textbox(label="Potássio ideal (mg/kg)"),
        gr.Textbox(label="Produção Máxima Prevista (ton/ha)")
    ],
    title="Otimização de Parâmetros Agrícolas",
    description="Escolha a cultura e o tipo de solo para calcular os valores ideais de PH, Umidade, Irrigação e Fertilizantes visando a máxima produção."
)

iface.launch(debug=True)

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


Keyboard interruption in main thread... closing server.


# README do Projeto: Otimização de Parâmetros Agrícolas

Este projeto demonstra um pipeline completo de Machine Learning para otimizar parâmetros agrícolas (PH, umidade, irrigação, e nutrientes NPK) visando maximizar a produção por hectare, utilizando dados de solo e cultura. O notebook culmina em uma interface interativa para recomendação de manejo agrícola.

## 1. Importação de Bibliotecas e Carregamento de Dados

**Objetivo**: Centralizar as dependências e carregar o dataset inicial.

- Importa bibliotecas essenciais como `pandas`, `gradio`, `numpy`, `xgboost`, `sklearn` (para modelagem e pré-processamento) e `scipy.optimize` (para otimização).
- Carrega o dataset `banco_solos_integrado_1000_linhas_v2.csv`.
- Realiza uma inspeção inicial (`head()`, `info()`, `describe()`) para entender a estrutura dos dados, tipos de variáveis e identificar valores ausentes.

## 2. Tratamento de Valores Ausentes

**Objetivo**: Preencher dados faltantes para garantir a integridade e usabilidade do dataset por algoritmos de Machine Learning.

- Variáveis categóricas (`cultura`, `tipo_solo`) são preenchidas com a **moda**.
- Variáveis numéricas são preenchidas com a **mediana**, escolhida por ser mais robusta a outliers em comparação com a média.

## 3. Codificação One-Hot

**Objetivo**: Converter variáveis categóricas em um formato numérico compreensível pelos modelos de Machine Learning.

- Aplica One-Hot Encoding nas colunas `cultura` e `tipo_solo` usando `pd.get_dummies()`.
- Utiliza `drop_first=True` para evitar multicolinearidade, tornando o modelo mais estável e interpretável.

## 4. Seleção de Features e Divisão Train/Test

**Objetivo**: Preparar os dados para o treinamento e avaliação do modelo.

- Define `producao_ton_ha` como a variável alvo (`y`).
- Remove colunas não preditivas (`id` e a coluna alvo) para formar o conjunto de features (`X`).
- Divide o dataset em conjuntos de treino e teste (`X_train`, `X_test`, `y_train`, `y_test`) usando `train_test_split` com `test_size=0.2` e `random_state=42` para reprodutibilidade.

## 5. Treinamento e Avaliação de Modelos

**Objetivo**: Treinar e comparar o desempenho de múltiplos algoritmos de regressão.

- Quatro modelos são treinados:
    - `LinearRegression`
    - `RandomForestRegressor`
    - `GradientBoostingRegressor`
    - `XGBRegressor`
- Para cada modelo, são calculadas as seguintes métricas de desempenho no conjunto de teste:
    - MAE (Mean Absolute Error)
    - MSE (Mean Squared Error)
    - RMSE (Root Mean Squared Error)
    - R² (R-squared)

## 6. Identificação do Melhor Modelo

**Objetivo**: Selecionar o modelo com o melhor desempenho para a etapa de otimização.

- O modelo com o maior valor de R² é identificado como o melhor, pois este coeficiente indica a proporção da variância da variável dependente que é explicável pelas variáveis independentes do modelo.
- O modelo selecionado (`best_model`) é armazenado para uso posterior.

## 7. Otimização de Parâmetros para Sugestão

**Objetivo**: Utilizar o modelo treinado para encontrar os parâmetros agrícolas ideais que maximizam a produção.

- Realiza uma **modelagem inversa**: ao invés de prever a produção, busca os inputs que maximizam a previsão de produção.
- Define as features de otimização (`ph`, `umidade_pct`, `irrigacao_horas`, `n_mgkg`, `p_mgkg`, `k_mgkg`).
- Define limites de otimização baseados nos valores mínimo e máximo observados no conjunto de treino (`feature_bounds`).
- Utiliza `scipy.optimize.minimize` com o método 'L-BFGS-B' para encontrar os valores ótimos. A função objetivo é o negativo da previsão do modelo para maximizar a produção.
- Exibe a produção máxima prevista e os valores ideais para cada parâmetro.

## 8. Interface Interativa com Gradio

**Objetivo**: Fornecer uma ferramenta prática e interativa para usuários finais, permitindo a seleção de cultura e tipo de solo para otimização personalizada.

- Uma função `otimizar_parametros` é criada, que aceita `cultura` e `tipo_solo` como entradas.
- Dentro desta função, a lógica de otimização é aplicada, ajustando as variáveis 'one-hot' correspondentes à cultura e tipo de solo selecionados.
- Os valores ideais de PH, umidade, irrigação e nutrientes NPK, juntamente com a produção máxima estimada, são retornados.
- Uma interface `gradio.Interface` é construída, permitindo aos usuários selecionar a cultura e o tipo de solo via dropdowns e visualizar os parâmetros ótimos em textboxes.
- A interface é lançada localmente, transformando o notebook em um protótipo de sistema de recomendação agrícola.

## Conclusão

Este projeto demonstra a aplicação prática de Machine Learning na agricultura, transformando dados brutos em insights acionáveis. Ao prever e otimizar as condições de cultivo, o modelo oferece uma ferramenta valiosa para agrônomos e produtores, permitindo a tomada de decisões mais informadas para maximizar a produtividade e a sustentabilidade das lavouras. A interface interativa com Gradio democratiza o acesso a essas recomendações, tornando a tecnologia de IA uma aliada estratégica no campo.